<a href="https://colab.research.google.com/github/hamagami/anomaly-detection/blob/main/04_06_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# オートエンコーダによる時系列異常検知

### 必要なモジュールのimport

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.models import Model


## 諸定義
### セグメントの切り出し関数

In [ ]:
from scipy import signal
#ヒストグラム　
def gethist(lst,range=(0,10),bins=25):
    win= signal.hann(len(lst)) 
    hist, bins= np.histogram(win*lst, bins, range)
    return hist, bins

In [ ]:
def segdata(lst, dim): #lst:１次元系列　dim:セグメントの幅
    segs = np.empty((0,dim), float)#0×dimの空配列
    hists = np.empty((0,25), float)#0×dimの空配列
    for i in range(lst.size - dim + 1):#1つづつずらしながらセグメントをつくっている。最後のセグメントの開始点は lst.size-dim
        hist,bins= gethist(lst[i:i+dim])
        
        seg=np.array(lst[i:i+dim][::-1].reshape((1,-1))) #セグメントの切り出し，時系列反転，appendのための2次ベクトル化
        hist=hist.reshape((1,-1))
        hists = np.append(hists,hist,axis=0)
        segs = np.append(segs,seg,axis=0)

    return segs, hists

### データ読み込み，パラメータ設定
 Keoghらの心電図のデータ  http://www.cs.ucr.edu/~eamonn/discords/qtdbsel102.txt
 Keogh, E., Lin, J. and Fu, A.: HOT SAX : Efficiently Finding the Most Unusual Time Series Subsequence, in Proceedings of the Fifth IEEE International Conference on Data Mining, ICDM 05, pp.226-233.

In [ ]:
def getdata():
  !wget "www.dropbox.com/s/x3fmb9mxr4xkip3/qtdbsel102.txt" #ローカルにコピーしてくる
  LEN=3000  #分析区間

  SP=0         #学習用データの開始点
  AP=3000   #テスト用データの開始点　個のデータの場合 4250ポイント付近に異常がある
  data = np.loadtxt("qtdbsel102.txt",delimiter="\t")
  print("データ数:",data.shape[0],"  次元数:",data.shape[1])

  #元データは3次元の時系列，3次のデータ(indexとしては2)を指定して学習/テストデータに分割
  train_org = data[SP:SP+LEN, 2]      #学習用データとして 1～2999サンプル区間を使用
  test_org  = data[AP:AP+LEN, 2]    #テスト用データとして3000～5999サンプルを使用
  
  #x軸
  x=np.arange(SP,SP+LEN)

  return x, train_org, test_org

In [ ]:
x, train_org, test_org = getdata()

plt.plot(x,train_org)
plt.title("train")
plt.legend()
plt.show()
plt.plot(x,test_org)
plt.title("test with anomaly")
plt.legend()
plt.show()

### 窓関数の設定と切り出し

In [ ]:
WLEN=256#セグメントのサイズ
train_seg, train_hist= segdata(train_org, WLEN)
test_seg, test_hist = segdata(test_org, WLEN)

## AEの学習(波形の場合）

In [ ]:
input_img = Input(shape=(WLEN,))
encoded = Dense(64, activation='linear')(input_img)
encoded = Dense(32, activation='relu')(encoded)
encoded = Dense(16, activation='relu',name='middle_code')(encoded)

decoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(decoded)
decoded = Dense(WLEN, activation='linear',name='output_code')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [ ]:
history=autoencoder.fit(train_seg, train_seg,
                epochs=1000,
                batch_size=50,
                shuffle=True,
                verbose=1,
                validation_split=0.1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['acc', 'val_acc'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['loss', 'val_loss'], loc='lower right')
plt.show()


### 復元誤差からの異常推定

In [ ]:
#encoded_sig = Model(input_img, encoded
decoded_trainsig = autoencoder.predict(train_seg)
decoded_testsig = autoencoder.predict(test_seg)
trainloss=[]
testloss=[]
for i,d in enumerate(decoded_trainsig):  
    d_trainsig = d-train_seg[i]
    d_testsig  = decoded_testsig[i]-test_seg[i]
    trainloss.append(np.linalg.norm(d_trainsig, ord=WLEN))
    testloss.append(np.linalg.norm(d_testsig, ord=WLEN))

plt.plot(train_org)
plt.title("train AE error")
outlier_rows = [i for i in range(len(trainloss)) if trainloss[i]>1]
for c in outlier_rows:
    plt.axvspan(c, c, color = "skyblue")
plt.show()

plt.plot(test_org)
plt.title("test AE error")
outlier_rows = [i for i in range(len(testloss)) if testloss[i]>1]
for c in outlier_rows:
    plt.axvspan(c, c, color = "coral")
plt.show()

##  AEの学習（ヒストグラム特徴の場合）

In [ ]:
# ヒストグラムの特徴は十分単純化されているため，AEをつかうと異常も復元できてしまうことがある。かなり低次圧縮して難しい問題にする
input_img = Input(shape=(25,))
encoded = Dense(32, activation='linear')(input_img)
encoded = Dense(16, activation='relu')(encoded)
encoded = Dense(8, activation='relu',name='middle_code')(encoded)

decoded = Dense(16, activation='relu')(encoded)
decoded = Dense(32, activation='relu')(decoded)
decoded = Dense(25, activation='linear',name='output_code')(decoded)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse',metrics=['accuracy'])

In [ ]:
history=autoencoder.fit(train_hist, train_hist,
                epochs=150,
                batch_size=50,
                shuffle=True,
                verbose=1,
                validation_split=0.1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_loss'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['acc', 'val_acc'], loc='lower right')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['loss', 'val_loss'], loc='lower right')
plt.show()

### 復元誤差からの異常検出


In [ ]:

decoded_trainsig = autoencoder.predict(train_hist)
decoded_testsig = autoencoder.predict(test_hist)
trainloss=[]
testloss=[]
for i,d in enumerate(decoded_trainsig):  
    d_trainsig = d-train_hist[i]
    d_testsig  = decoded_testsig[i]-test_hist[i]
    trainloss.append(np.linalg.norm(d_trainsig, ord=WLEN))
    testloss.append(np.linalg.norm(d_testsig, ord=WLEN))

plt.plot(trainloss)
plt.show()
plt.plot(testloss)
plt.show()
plt.show()
plt.plot(train_org)
plt.title("train AE error")
outlier_rows = [i for i in range(len(trainloss)) if trainloss[i]>15]
for c in outlier_rows:
    plt.axvspan(c, c, color = "skyblue")
plt.show()

plt.plot(test_org)
plt.title("test AE error")
outlier_rows = [i for i in range(len(testloss)) if testloss[i]>15]
for c in outlier_rows:
    plt.axvspan(c, c, color = "coral")
plt.show()